In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
data_dir = r'E:\crop-disease-prediction\PlantVillage'

In [4]:
# Image dimensions
img_height, img_width = 150, 150

In [9]:
datagen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Set validation split
)

In [6]:
datagen_test = ImageDataGenerator(rescale=1./255)

In [10]:
train_generator = datagen_train.flow_from_directory(
    directory=data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

Found 3703 images belonging to 5 classes.


In [11]:
validation_generator = datagen_train.flow_from_directory(
    directory=data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

Found 924 images belonging to 5 classes.


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')  # Output layer
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

c:\Program Files\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator
)

Epoch 1/20


c:\Program Files\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


116/116 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - accuracy: 0.4421 - loss: 1.3908 - val_accuracy: 0.7284 - val_loss: 0.7730
Epoch 2/20
116/116 ━━━━━━━━━━━━━━━━━━━━ 72s 600ms/step - accuracy: 0.7590 - loss: 0.6898 - val_accuracy: 0.8626 - val_loss: 0.4240
Epoch 3/20
116/116 ━━━━━━━━━━━━━━━━━━━━ 65s 553ms/step - accuracy: 0.8532 - loss: 0.4412 - val_accuracy: 0.8561 - val_loss: 0.3867
Epoch 4/20
116/116 ━━━━━━━━━━━━━━━━━━━━ 65s 550ms/step - accuracy: 0.8720 - loss: 0.3866 - val_accuracy: 0.9069 - val_loss: 0.2571
Epoch 5/20
116/116 ━━━━━━━━━━━━━━━━━━━━ 64s 546ms/step - accuracy: 0.9061 - loss: 0.2775 - val_accuracy: 0.9318 - val_loss: 0.2159
Epoch 6/20
116/116 ━━━━━━━━━━━━━━━━━━━━ 66s 562ms/step - accuracy: 0.9243 - loss: 0.2242 - val_accuracy: 0.9264 - val_loss: 0.1981
Epoch 7/20
116/116 ━━━━━━━━━━━━━━━━━━━━ 65s 554ms/step - accuracy: 0.9248 - loss: 0.2154 - val_accuracy: 0.9600 - val_loss: 0.1242
Epoch 8/20
116/116 ━━━━━━━━━━━━━━━━━━━━ 65s 550ms/step - accuracy: 0.9415 - loss: 0.1737 - val_a

In [14]:
test_loss, test_acc = model.evaluate(validation_generator)
print(f"Test accuracy: {test_acc * 100:.2f}%")

29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 193ms/step - accuracy: 0.9790 - loss: 0.0650
Test accuracy: 97.94%


In [26]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_disease(img_path):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Create batch axis
    predictions = model.predict(img_array)
    class_names = list(train_generator.class_indices.keys())
    predicted_class = class_names[np.argmax(predictions)]
    return predicted_class

# Example usage
print(predict_disease(r'PlantVillage/Potato___Late_blight/0b2bdc8e-90fd-4bb4-bedb-485502fe8a96___RS_LB 4906.JPG'))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Pepper__bell___healthy


In [29]:
# Save the model
model.save('model.h5')  # Save in HDF5 format